### IMPORT LIBRARIES & CREATE CONNECTOR

In [1]:
import pandas as pd
import numpy as np
import sqlalchemy as sa
import mysql.connector
import getpass

In [2]:
HOST = 'localhost'
USER = 'root'
DATABASE = 'goodreads'
PASSWORD = getpass.getpass(f'Enter password for {USER}: ')

In [3]:
def getconn():
    conn = mysql.connector.connect(
        host=HOST,
        user=USER,
        password=PASSWORD,
        database=DATABASE
    )
    return conn

pool = sa.create_engine(
    "mysql+mysqlconnector://",
    creator=getconn,
)

with pool.connect() as db_conn:
    results = db_conn.execute(sa.text("SELECT NOW()")).fetchone()
    print("Current time: ", results[0])

Current time:  2023-05-25 12:18:31


### SANDBOX

In [4]:
book_id_query = sa.text(
    "SELECT goodreads_book_id FROM books;"
)
book_id = pd.read_sql_query(book_id_query, con=pool.connect())
book_id.head()

,goodreads_book_id
0,2767052
1,3
2,41865
3,2657
4,4671


In [5]:
id_list = book_id.goodreads_book_id.tolist()

In [6]:
user_id_query = sa.text(
    "SELECT DISTINCT(user_id) FROM new_ratings_;"
)
user_id = pd.read_sql_query(user_id_query, con=pool.connect())
user_id.head()

,user_id
0,1
1,10
2,100
3,1000
4,10000


In [7]:
user_list = user_id.user_id.tolist()
user_list[:5]

['1', '10', '100', '1000', '10000']

In [8]:
def get_ratings(user_id):
    query = sa.text(
        f"SELECT goodreads_book_id as book_id, rating FROM new_ratings_ WHERE user_id = {user_id};"
    )
    result = pd.read_sql_query(query, con=pool.connect())
    books_list = result.book_id.tolist()
    ratings_list = result.rating.apply(lambda x: round(x*0.2,4)).tolist()
    return books_list, ratings_list

In [21]:
user_ratings_df = pd.DataFrame(columns=list(['user_id']+id_list))
for user in user_list[:5]:
    user_books, user_ratings = get_ratings(user)
    user_profile = pd.DataFrame({'user_id': [user]}, columns=user_ratings_df.columns)
    user_profile.loc[0,user_books] = user_ratings
    user_profile.fillna(0, inplace=True)
    user_ratings_df = pd.concat([user_ratings_df, user_profile], ignore_index=True)

In [17]:
len(user_list)

53424

In [22]:
user_ratings_df

,user_id,2767052,3,41865,2657,4671,11870085,5907,5107,960,...,101094,13616278,4936457,4769651,15613,7130616,208324,77431,8565083,8914
0,1,0,0,0,1.0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,10,0,0,0,1.0,1.0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,100,0,0,0,0.0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,1000,0,0,0,0.0,0.2,0,0.6,0.4,0,...,0,0,0,0,0,0,0,0,0,0
4,10000,0,1.0,0.6,0.8,0,0,0,0,0.8,...,0,0,0,0,0,0,0,0,0,0
